# Zadanie 5 - Algorytm genetyczny

Implement a genetic algorithm to schedule jobs using available resources, so that the time to finish all tasks is minimal. Test your algorithm using different parameters and analyse the results. The data is in a separate excel files – each task has the resource (R) and time needed using this resource (T) in a specific order. You cannot change the order of operations within the task.

## Generowanie problemu

In [1]:
import pandas as pd
import numpy as np
import random

In [ ]:
excel_data = pd.read_excel(r"GA_task.xlsx")

In [3]:
excel_data

,1,Unnamed: 1,2,Unnamed: 3,3,Unnamed: 5,4,Unnamed: 7,5,Unnamed: 9,...,46,Unnamed: 91,47,Unnamed: 93,48,Unnamed: 95,49,Unnamed: 97,50,Unnamed: 99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27


In [4]:
class subtask():
    def __init__(self, machine, duration, parent):
        self.machine = machine-1
        self.duration = duration
        self.parent = parent
        self.child = None
        
    def assign_child(self, child):
        self.child = child
    
    def assign_time(self, time):
        self.time = time
        self.time_and_duration = self.time + self.duration
        
        return self
    
    
    def __repr__(self):
        return 'task on machine ' + str(self.machine) + ' with duration ' + str(self.duration)

In [ ]:
class Task():
    
    def __init__(self, name):
        self.name = name
        self.last_placed = 0
    
    def generate_subtasks(self, table):
        list_of_subs = []
        
        for i in range(11):
            
            machine = table.iloc[i+1].get(table.columns[0])
            dur = table.iloc[i+1].get(table.columns[1])
            
            if i==0:
                task_to_list = subtask(machine, dur, None)
            else:
                task_to_list = subtask(machine, dur, list_of_subs[-1])
                list_of_subs[-1].assign_child(task_to_list)
                
            list_of_subs.append(task_to_list)
        
        self.subtasks = list_of_subs
        
        return list_of_subs
    
    def return_last_not_placed(self):
        
        item = self.subtasks[self.last_placed]
        self.last_placed = self.last_placed + 1
        
        if self.last_placed == 11:
            self.last_placed = 0
        
        return item

     
    
    def __repr__(self):
        return self.name

Generowanie listy z zadaniami z podzadaniami

In [ ]:
tasks = []

for i in range(50):
    taskk = Task('task'+str(i+1))
    taskk.generate_subtasks(excel_data.iloc[:, 2*i:2*i+2])
    tasks.append(taskk)
    
print(tasks)    

[task1, task2, task3, task4, task5, task6, task7, task8, task9, task10, task11, task12, task13, task14, task15, task16, task17, task18, task19, task20, task21, task22, task23, task24, task25, task26, task27, task28, task29, task30, task31, task32, task33, task34, task35, task36, task37, task38, task39, task40, task41, task42, task43, task44, task45, task46, task47, task48, task49, task50]


Metody:

- crossover 

- mutation - zamiana dwóch zadań, tak aby nie zaburzyć kolejności 

Funkcja fitness: czas od pierwszego uruchomionego zadania do końca ostatniego

Reprezentacja chromosomu - macierz - wiersze - maszyny, kolumny - czas

Generowanie populacji

In [7]:
def generate_population(n_population):
    population = [[random.uniform(0, 1) for j in range(50)] for i in range(n_population)]
    
    return population

In [8]:
def mutation(individual):
    random_number = random.randint(0, len(individual)-1)
    
    #min value = 0; max value = 1
    individual[random_number] += random.uniform(-individual[random_number], 1-individual[random_number])
    
    return individual

In [9]:
def crossover(individual1, individual2):
    random_number = random.randint(1, len(individual1)-2)

    return individual1[:random_number]+individual2[random_number:], individual2[:random_number]+individual1[random_number:]

In [10]:
def sort_dict_by_key(d):
    #segreguje słownik malejąco
    return dict(sorted(d.items()), reverse=True)

In [11]:
def decode_chromosome(chromosome, list_of_tasks):
    
    #każda wartość to czas pracy dla maszyny
    #10 maszyn
    factory = [0 for i in range(10)]
    
    #kluczami są wartości chromosomu, a wartościami odpowiadające jej zadanie
    priority = dict()
    
    for num in range(len(chromosome)):
        priority[chromosome[num]] = list_of_tasks[num]
        
    priority = sort_dict_by_key(priority)
    
    chromosome_sorted = sorted(chromosome, reverse=True)
    
    for i in range(11):
        for value in chromosome_sorted:
            
            subtask_to_be_placed = priority[value].return_last_not_placed()
            machine_to_place = subtask_to_be_placed.machine
            duration_of_subtask = subtask_to_be_placed.duration
            
            if subtask_to_be_placed.parent == None:
                subtask_to_be_placed.assign_time(factory[machine_to_place])
                factory[machine_to_place] = factory[machine_to_place] + duration_of_subtask
            
            elif subtask_to_be_placed.parent.time_and_duration > factory[machine_to_place]:
                subtask_to_be_placed.assign_time(subtask_to_be_placed.parent.time_and_duration)
                factory[machine_to_place] = subtask_to_be_placed.time_and_duration + duration_of_subtask
            
            else:
                subtask_to_be_placed.assign_time(factory[machine_to_place])
                factory[machine_to_place] = factory[machine_to_place] + duration_of_subtask     
            
    return factory

In [12]:
def calculate_fitness(decoded_chromosome):
    return max(decoded_chromosome)

In [ ]:
tasks = []

for i in range(50):
    taskk = Task('task'+str(i+1))
    taskk.generate_subtasks(excel_data.iloc[:, 2*i:2*i+2])
    tasks.append(taskk)
    
print(tasks) 

[task1, task2, task3, task4, task5, task6, task7, task8, task9, task10, task11, task12, task13, task14, task15, task16, task17, task18, task19, task20, task21, task22, task23, task24, task25, task26, task27, task28, task29, task30, task31, task32, task33, task34, task35, task36, task37, task38, task39, task40, task41, task42, task43, task44, task45, task46, task47, task48, task49, task50]


In [16]:
def algorithm_with_elitism(tasks_list, prop_mutation=0.15, num_population=1000, n_iter=600):
    #initialization
    population = generate_population(num_population)
    population.sort(key=lambda x: calculate_fitness(decode_chromosome(x, tasks_list)))
    print('Wynik na początku:', calculate_fitness(decode_chromosome(population[0], tasks_list)))
    
    for i in range(n_iter):
        new_population = []
        for num in range(int(num_population/2)):
            random_num = random.uniform(0, 1)
            if random_num<prop_mutation:
                offspring1 = mutation(population[2*num])
                offspring2 = mutation(population[2*num+1])
                new_population.append(offspring1)
                new_population.append(offspring2)
            else:
                offspring1, offspring2 = crossover(population[2*num], population[2*num+1])
                new_population.append(offspring1)
                new_population.append(offspring2)
        
        new_population.sort(key=lambda x: calculate_fitness(decode_chromosome(x, tasks_list)))
        population.sort(key=lambda x: calculate_fitness(decode_chromosome(x, tasks_list)))    
        population = new_population[:int(num_population/2)] + population[:int(num_population/2)]
        random.shuffle(population)
        
    decoded_population = [decode_chromosome(pop, tasks_list) for pop in population]       
    
    scores = [calculate_fitness(individual) for individual in decoded_population]
    score = min(scores)
    number_of_population = np.argmin(scores)
    
    print('Wynik na koniec:', score)
    
    return population[number_of_population], score

In [ ]:
tasks = []

for i in range(50):
    taskk = Task('task'+str(i+1))
    taskk.generate_subtasks(excel_data.iloc[:, 2*i:2*i+2])
    tasks.append(taskk)
    
print(tasks) 

[task1, task2, task3, task4, task5, task6, task7, task8, task9, task10, task11, task12, task13, task14, task15, task16, task17, task18, task19, task20, task21, task22, task23, task24, task25, task26, task27, task28, task29, task30, task31, task32, task33, task34, task35, task36, task37, task38, task39, task40, task41, task42, task43, task44, task45, task46, task47, task48, task49, task50]


In [21]:
result2 = algorithm_with_elitism(tasks, prop_mutation=0.85, num_population=500, n_iter=500)

Wynik na początku: 2297
Wynik na koniec: 2231


In [22]:
result3 = algorithm_with_elitism(tasks, prop_mutation=0.15, num_population=500, n_iter=500)

Wynik na początku: 2312
Wynik na koniec: 2223


In [23]:
result4 = algorithm_with_elitism(tasks, prop_mutation=0.15, num_population=100, n_iter=100)

Wynik na początku: 2332
Wynik na koniec: 2227


In [24]:
result5 = algorithm_with_elitism(tasks, prop_mutation=0.85, num_population=100, n_iter=100)

Wynik na początku: 2355
Wynik na koniec: 2257


In [25]:
result6 = algorithm_with_elitism(tasks, prop_mutation=0.15, num_population=500, n_iter=100)

Wynik na początku: 2296
Wynik na koniec: 2223
